In [ ]:
#importing necessary libraries
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import roc_curve, auc

import re

import pickle
from tqdm import tqdm
import os



In [ ]:
#loading dataset
data=pd.read_csv("../input/womens-ecommerce-clothing-reviews/Womens Clothing E-Commerce Reviews.csv")


In [ ]:
#what is the shape of the dta set?
data.shape

In [ ]:

#which columns are there?
data.columns

In [ ]:
#lets see the first 5 rows of each column.

data.head(5)

!].We found some NaN values. So we will drop that in the next step.

In [ ]:
#dropping nan values
data=data.dropna()
data.head(2)

We have unnamed: 0 column which is unnecessary.

In [ ]:
#dropping unnamed column
data=data.drop(["Unnamed: 0"],axis=1).reset_index()
data.head(2)

In [ ]:
#Do you want to see more information about data set?...............

In [ ]:
#here it is........
data.info()

We don't have any null values... great!!!!

In [ ]:
#Lets describe some statistical data.......

In [ ]:
data.describe()

In [ ]:
#lets check how many different values are there for clothing id
data["Clothing ID"].value_counts()

In [ ]:
#lets get the value counts of division name
data["Division Name"].value_counts()

We have only 3 Divisions.. 

In [ ]:
#lets check the value counts of department name
data["Department Name"].value_counts()

In [ ]:
#lets check the value counts of class name
data["Class Name"].value_counts()

***Okayy.. Now lets check our class label i.e. Recommended IND, also % of recommendations........***

In [ ]:
a=data["Recommended IND"].value_counts()
print(a)
print("The products that are recommended are : ", (a[1]/(a[0]+a[1]))*100,"%")
print("the productys that are not recommended are :", (a[0]/(a[0]+a[1]))*100,"%")

***The dataset is imbalanced. 1 means recommended. 0 means not recommended.***<br>
1]. We have ***81.8%*** of ***recommended*** data points.... and only ***18.18%*** of ***not recommended*** data points...
    

# Let,s analyse.. (UNIVARIATE ANALYSIS)

In [ ]:
#lets define function for stack plot..........

In [ ]:
def stack_plot(data,xtick,col2="Recommended IND",col3="total"):
    ind=np.arange(data.shape[0])
    
    plt.figure(figsize=(25,10))
    
    p1=plt.bar(ind,data[col3].values)
    p2=plt.bar(ind,data[col2].values)
    
    plt.ylabel("Recommendation")
    plt.xticks(ind,list(data[xtick].values))
    plt.legend((p1[0],p2[0]),("total","recommended"))
    plt.show()
    
    

In [ ]:
def univariate(data,col1,col2="Recommended IND",top=False):
    temp=pd.DataFrame(data.groupby(col1)[col2].agg(lambda x:x.eq(1).sum())).reset_index()
    
    recommend=temp[col2]
    temp=pd.DataFrame(data.groupby(col1)[col2].agg([("Avg","mean"),("total","count")]).reset_index())
    temp[col2]=recommend
    
    temp.sort_values(by=["total"],inplace=True,ascending=False)
    
    if top:
        temp=temp[0:top]
        
    stack_plot(temp,xtick=col1,col2=col2,col3="total")
    print(temp.head(5))
    print("*"*50)
    print(temp.tail(5))

In [ ]:
univariate(data,"Clothing ID","Recommended IND",False)

We can observe that, clothing_id 1078 is recommended 707 times out of 871 reviews.. <br>similarly Clothing_id 862 is recommended 534 times out of 658 reviews

In [ ]:
univariate(data,"Division Name","Recommended IND",False)

We can observe that General division dresses recommended 9490 times out of 11664 of general dress reviews.

In [ ]:
univariate(data,"Department Name","Recommended IND",False)

Tops are recommended 7047 times out of 8713 tops reviews

In [ ]:
univariate(data,"Class Name","Recommended IND",False)

Dresses recommended 4314 times out of 5371 dresses reviews

In [ ]:
univariate(data,"Rating","Recommended IND",False)

We can observe that the clothings which are rated 5 are highly recommended.. 10837 clothings which rated 5 are recommended out of 10858..<br>
ratings with 1 are not much recommended. only 7 clothings which are rated 1 are recommended out of 691..<br>***So we can say the higher the rating higher is the recommendation....***

In [ ]:
univariate(data,"Positive Feedback Count","Recommended IND",False)

In [ ]:
univariate(data,"Age","Recommended IND",False)

From this we could say that reviewers of age 39 recommended 962 clothings out of 1103. and they are the people who shopped most.

In [ ]:
#How to calculate number of words in a string in DataFrame: https://stackoverflow.com/a/37483537/4084039
word_count = data['Title'].str.split().apply(len).value_counts()
word_dict = dict(word_count)
word_dict = dict(sorted(word_dict.items(), key=lambda kv: kv[1]))


ind = np.arange(len(word_dict))
plt.figure(figsize=(20,5))
p1 = plt.bar(ind, list(word_dict.values()))

plt.ylabel('total')
plt.title('Words for each title')
plt.xticks(ind, list(word_dict.keys()))
plt.show()

Most of the title contain only two words.. second highest is 3 words..

# Cleaning the data......

In [ ]:
#convert all letters to lower case, replace the space by _ 
data["Class Name"]=data["Class Name"].str.lower()
data["Class Name"]=data["Class Name"].str.replace(" ","_")

#Want to see the va;ue_counts??
data["Class Name"].value_counts()

In [ ]:
data["Department Name"]=data["Department Name"].str.lower()
data["Department Name"]=data["Department Name"].str.replace(" ","_")
data["Department Name"].value_counts()

In [ ]:
data["Division Name"]=data["Division Name"].str.lower()
data["Division Name"]=data["Division Name"].str.replace(" ","_")
data["Division Name"].value_counts()

In [ ]:
#let us define a function to replace shorthand notations with full....

In [ ]:
# https://stackoverflow.com/a/47091490/4084039
import re

def decontracted(phrase):
    # specific
    phrase = re.sub(r"won't", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase

In [ ]:
# https://gist.github.com/sebleier/554280
# we are removing the words from the stop words list: 'no', 'nor', 'not'
stopwords= ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've",\
            "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', \
            'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their',\
            'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', \
            'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', \
            'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', \
            'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after',\
            'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further',\
            'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more',\
            'most', 'other', 'some', 'such', 'only', 'own', 'same', 'so', 'than', 'too', 'very', \
            's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', \
            've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn',\
            "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn',\
            "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", \
            'won', "won't", 'wouldn', "wouldn't"]

In [ ]:
# Combining all the above......
from tqdm import tqdm
def preprocess_text(text_data):
    preprocessed_text = []
    # tqdm is for printing the status bar
    for sentance in tqdm(text_data):
        sent = decontracted(sentance)
        sent = sent.replace('\\r', ' ')
        sent = sent.replace('\\n', ' ')
        sent = sent.replace('\\"', ' ')
        sent = re.sub('[^A-Za-z0-9]+', ' ', sent)
        # https://gist.github.com/sebleier/554280
        sent = ' '.join(e for e in sent.split() if e.lower() not in stopwords)
        preprocessed_text.append(sent.lower().strip())
    return preprocessed_text

In [ ]:
preprocessed_title=preprocess_text(data["Title"].values)

In [ ]:
#lets replace the title column in the dataframe with preprocessed title.
data["Title"]=preprocessed_title

In [ ]:
#lets see how the review text looks like before preprocessing..(here we can see the 2nd review)
data["Review Text"][1]

In [ ]:
preprocessed_review=preprocess_text(data["Review Text"].values)

In [ ]:
preprocessed_review[1]

In [ ]:
#lets replace the review column in the dataframe with preprocessed review.
data["Review Text"]=preprocessed_review

In [ ]:
#Lets take out our class label (recommended ind) and keep it in variable y..
# because this is the class label that we have to predict given all other features.
#and all other features are kept in x variable..
y=data["Recommended IND"]
x=data.drop(["Recommended IND"],axis=1)
x.head(1)

In [ ]:
#NOW, split data into train,cv,test data.......
from sklearn.model_selection import train_test_split
X_train,x_test,Y_train,y_test=train_test_split(x,y,test_size=0.33,stratify=y, random_state =41)
x_train,x_cv,y_train,y_cv=train_test_split(X_train,Y_train,test_size=0.33,stratify=Y_train, random_state =41)

# PRE_PROCESSING.......

# Bag Of Words(BOW)

In [ ]:
vec=CountVectorizer()
vec.fit(x_train["Title"].values)
print(x_train.shape,y_train.shape)

x_train_title=vec.transform(x_train["Title"].values)
x_cv_title=vec.transform(x_cv["Title"].values)
x_test_title=vec.transform(x_test["Title"].values)

print("after vectorization....")
print(x_train_title.shape,y_train.shape)
print(x_cv_title.shape,y_cv.shape)
print(x_test_title.shape,y_test.shape)

In [ ]:
vec=CountVectorizer()
vec.fit(x_train["Review Text"].values)

x_train_rev=vec.transform(x_train["Review Text"].values)
x_cv_rev=vec.transform(x_cv["Review Text"].values)
x_test_rev=vec.transform(x_test["Review Text"].values)

print("after vectorization....")
print(x_train_rev.shape,y_train.shape)
print(x_cv_rev.shape,y_cv.shape)
print(x_test_rev.shape,y_test.shape)

In [ ]:
vec=CountVectorizer()
vec.fit(x_train["Division Name"].values)

x_train_div=vec.transform(x_train["Division Name"].values)
x_cv_div=vec.transform(x_cv["Division Name"].values)
x_test_div=vec.transform(x_test["Division Name"].values)

print("after vectorization....")
print(x_train_div.shape,y_train.shape)
print(x_cv_div.shape,y_cv.shape)
print(x_test_div.shape,y_test.shape)

print(vec.get_feature_names())

In [ ]:
vec=CountVectorizer()
vec.fit(x_train["Department Name"].values)

x_train_dep=vec.transform(x_train["Department Name"].values)
x_cv_dep=vec.transform(x_cv["Department Name"].values)
x_test_dep=vec.transform(x_test["Department Name"].values)

print("after vectorization....")
print(x_train_dep.shape,y_train.shape)
print(x_cv_dep.shape,y_cv.shape)
print(x_test_dep.shape,y_test.shape)

print(vec.get_feature_names())

In [ ]:
vec=CountVectorizer()
vec.fit(x_train["Class Name"].values)

x_train_cls=vec.transform(x_train["Class Name"].values)
x_cv_cls=vec.transform(x_cv["Class Name"].values)
x_test_cls=vec.transform(x_test["Class Name"].values)

print("after vectorization....")
print(x_train_cls.shape,y_train.shape)
print(x_cv_cls.shape,y_cv.shape)
print(x_test_cls.shape,y_test.shape)

print(vec.get_feature_names())


In [ ]:
#We keep our rating data which is categorical data in numerical form as it is.

In [ ]:
x_train_rate=(np.array(x_train["Rating"])).reshape((-1,1))
x_cv_rate=(np.array(x_cv["Rating"])).reshape(-1,1)
x_test_rate=(np.array(x_test["Rating"])).reshape((-1,1))


print("after vectorization.......")
print(x_train_rate.shape,y_train.shape)
print(x_cv_rate.shape,y_cv.shape)
print(x_test_rate.shape,y_test.shape)


In [ ]:
#normalize age column.
from sklearn.preprocessing import Normalizer
norm=Normalizer()

norm.fit(x_train["Age"].values.reshape(1,-1))

x_train_age=norm.transform(x_train["Age"].values.reshape(1,-1))
x_cv_age=norm.transform(x_cv["Age"].values.reshape(1,-1))
x_test_age=norm.transform(x_test["Age"].values.reshape(1,-1))

x_train_age=x_train_age.reshape(-1,1)
x_cv_age=x_cv_age.reshape(-1,1)
x_test_age=x_test_age.reshape(-1,1)


print("after vectorization.......")
print(x_train_age.shape,y_train.shape)
print(x_cv_age.shape,y_cv.shape)
print(x_test_age.shape,y_test.shape)


In [ ]:
#Combining all the preprocessed data

In [ ]:
from scipy.sparse import hstack

x_tr=hstack((x_train_rev,x_train_title,x_train_div,x_train_dep,x_train_cls,x_train_rate,x_train_age)).tocsr()
x_cv=hstack((x_cv_rev,x_cv_title,x_cv_div,x_cv_dep,x_cv_cls,x_cv_rate,x_cv_age)).tocsr()
x_te=hstack((x_test_rev,x_test_title,x_test_div,x_test_dep,x_test_cls,x_test_rate,x_test_age)).tocsr()

print("FINAL DATA MATRIX SHAPE IS ........")
print(x_tr.shape,y_train.shape)
print(x_cv.shape,y_cv.shape)
print(x_te.shape,y_test.shape)
print("*"*100)

In [ ]:
#This function takes classifier and data , and then gives probability_score of class label

In [ ]:
def batch_predict(clf, data):
    """This function takes classifier and data , and then gives probability_score of class label"""
    y_data_pred = []                                                 #list to store probability of class
    tr_loop = data.shape[0] - data.shape[0]%1000                     #to loop through batchwise and took 1000 for batch
    
    for i in range(0, tr_loop, 1000):
        y_data_pred.extend(clf.predict_proba(data[i:i+1000])[:,1])     # this loop for upto last 1000 multiplier
    
    if data.shape[0]%1000 !=0:                                        #predicting for last data
        y_data_pred.extend(clf.predict_proba(data[tr_loop:])[:,1])
    
    return y_data_pred

In [ ]:
import math
import matplotlib.pyplot as plt
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import roc_auc_score

train_auc = []    
cv_auc = []
alpha = [0.00001,0.0001,0.001,0.01,0.1,1,10,50,100]         #to find the best alpha value
for i in tqdm(alpha):
    neigh = MultinomialNB(alpha=i,class_prior=[0.5,0.5],fit_prior=False)
    neigh.fit(x_tr, y_train)

    y_train_pred = batch_predict(neigh, x_tr)               #finding probability scores of x_tr using batch_predict function
    y_cv_pred = batch_predict(neigh, x_cv)                  ##finding probability scores of x_cv using batch_predict function

          
    train_auc.append(roc_auc_score(y_train,y_train_pred))   #appending auc values
    cv_auc.append(roc_auc_score(y_cv, y_cv_pred))


    
plt.plot(alpha, train_auc, label='Train AUC')        #plotting hyper_parameter v/s auc value plot....
plt.plot(alpha, cv_auc, label='CV AUC')
plt.xscale("log")
plt.scatter(alpha , train_auc, label='Train AUC points')
plt.scatter(alpha , cv_auc, label='CV AUC points')

plt.legend()
plt.xlabel("log(alpha): hyperparameter")
plt.ylabel("AUC")
plt.title("hyperparameter v/s AUC plot to find BEST ALPHA")
plt.grid()
plt.show()

In [ ]:
#best alpha = 10^0 = 1
best_alpha_bow=1

In [ ]:
# https://scikit-learn.org/stable/modules/generated/sklearn.metrics.roc_curve.html#sklearn.metrics.roc_curve
from sklearn.metrics import roc_curve, auc


clas = MultinomialNB(alpha=best_alpha_bow,class_prior=[0.5,0.5],fit_prior=False)
clas.fit(x_tr, y_train)


y_train_pred = batch_predict(clas, x_tr)    
y_test_pred = batch_predict(clas, x_te)

# roc_auc_score(y_true, y_score) the 2nd parameter should be probability estimates of the positive class
# not the predicted outputs
train_fpr, train_tpr, tr_thresholds = roc_curve(y_train, y_train_pred)
test_fpr, test_tpr, te_thresholds = roc_curve(y_test, y_test_pred)


train_auc=auc(train_fpr, train_tpr)
test_auc=auc(test_fpr, test_tpr)
plt.plot(train_fpr, train_tpr, label="train AUC ="+str(train_auc))
plt.plot(test_fpr, test_tpr, label="test AUC ="+str(test_auc))
plt.legend()
plt.xlabel("False positive rate")
plt.ylabel("True positive rate")
plt.title("FPR v/s TPR plot")
plt.grid()
plt.show()

In [ ]:
def find_best_threshold(threshold, fpr, tpr):
    """it will give best threshold value that will give the least fpr"""
    t = threshold[np.argmax(tpr*(1-fpr))]
    
    # (tpr*(1-fpr)) will be maximum if your fpr is very low and tpr is very high
    print("the maximum value of tpr*(1-fpr)", max(tpr*(1-fpr)), "for threshold", np.round(t,3))
    
    return t

def predict_with_best_t(proba, threshold):
    """this will give predictions based on best threshold value"""
    predictions = []
    for i in proba:
        if i>=threshold:
            predictions.append(1)
        else:
            predictions.append(0)
    return predictions

In [ ]:
#computing confusion matrix for set_1
from sklearn.metrics import confusion_matrix
best_t = find_best_threshold(tr_thresholds, train_fpr, train_tpr)
print("Train confusion matrix")
m_tr=(confusion_matrix(y_train, predict_with_best_t(y_train_pred, best_t)))
print(m_tr)
print("Test confusion matrix")
m_te=(confusion_matrix(y_test, predict_with_best_t(y_test_pred, best_t)))
print(m_te)
#you are concerned that this cell produced fifferent result?
#yes sir
#no. if i run this code from the begining then it will be different result

From the confusion matrix , we can observe that, TPR, TNR are high. FPR,FNR are significantly lower.<br>***So We got a good result***

In [ ]:
cm_tr = np.array(m_tr)
cm_te=np.array(m_te)
#plotting heatmap
sns.heatmap(cm_tr, annot=True,fmt="d",cmap='Blues')
plt.xlabel("predict")
plt.ylabel("actual")
plt.show()

In [ ]:
sns.heatmap(cm_te,annot=True,fmt="d",cmap="BrBG")
plt.xlabel("predict")
plt.ylabel("actual")
plt.show()

We can see the result of confusion matrix through ***HEAT MAP*** also.

In [ ]:
#Lets calculate precision,recall, and f1 score

precision= m_te[1][1]/(m_te[1][0]+m_te[1][1])
print(precision)

recall=m_te[1][1]/(m_te[0][1]+m_te[1][1])
print(recall)

f1=(2*precision*recall)/(precision+recall)
print(f1)

In [ ]:
index=[1]
summary_df=pd.DataFrame({"Vectorizer":("BOW"),"Hyper-parameter(alpha)":(best_alpha_bow),"Train_AUC":(train_auc),"Test_AUC":(test_auc), "Precision":(precision),"Recall":(recall),"F_1_Score":(f1)},index=index)
summary_df

1]. When we use BOW vectorizer , we got 0.95 as test AUC.<br>     This means there is 95% chance that our model will be able to classify positive and negative points i.e. ***recommended*** or ***not recommended***<br>2]. We got TPR and TNR high, and FPR,FNR lower, this shows our model predicts pretty well.<br>